# Can we classify open clusters and globular clusters using their B-V index and true diameter?

## Abstract

## Introduction

## Reading the Data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm


In [2]:
def get_angular_size(r_h):
	'''Go  from half light radius to true radius, based on King (1966) model for half light radius.
	Takes half light radius, in arcmin, and multiply by 2.5 to get angular radius.
	Multiply by 2 again to get angular diameter (angular size)
	'''
	return 2.5 * r_h * 2


def get_true_diameter(angular_size, dist):
	'''Takes angular size, in arcmin, and distance from Sun, in pc,
	Calculates true diameter (in pc)
	'''
	diam_rad = angular_size * (2.9089 * 10**-4) # 2.9089 * 10^-4 rad = 1 arcmin
	return diam_rad * dist 


In [3]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 


def get_angular_size(r_h):
	'''Go  from half light radius to true radius, based on King (1966) model for half light radius.
	Takes half light radius, in arcmin, and multiply by 2.5 to get angular radius.
	Multiply by 2 again to get angular diameter (angular size)
	'''
	return 2.5 * r_h * 2


def get_true_diameter(angular_size, dist):
	'''Takes angular size, in arcmin, and distance from Sun, in pc,
	Calculates true diameter (in pc)
	'''
	diam_rad = angular_size * (2.9089 * 10**-4) # 2.9089 * 10^-4 rad = 1 arcmin
	return diam_rad * dist 


# GLOBULAR CLUSTER DATA READING
glob_data1 = pd.read_csv("glob_data.txt", skiprows = 72, nrows = 157, delim_whitespace = True, header = None)
glob_data2 = pd.read_csv("glob_data.txt", skiprows = 252, nrows = 157, delim_whitespace = True, header = None)
glob_data3 = pd.read_csv("glob_data.txt", skiprows = 433, nrows = 157, delim_whitespace = True, header = None)

# Combining the first two columns because the names were split
glob_data1["Cname"] = glob_data1.iloc[:,0] + glob_data1.iloc[:,1].astype(str)


glob_data = pd.concat([glob_data1["Cname"], glob_data1.iloc[:,4:10], glob_data1.iloc[:,10], glob_data2.iloc[:,4], glob_data3.iloc[:,9]], axis = 1)
glob_data.columns = ["Cname", "RA1", "RA2","RA3", "Dec1", "Dec2", "Dec3", "R_sun", "E(B-V)","r_h" ]
glob_data = glob_data.dropna()

# OPEN CLUSTER DATA READING
open_data = pd.read_csv("open_data.tsv", skiprows = 39, delimiter = "\t", header = None)

# Code generated by ChatGPT-4o (OpenAI, 2024) - November 2024 version, Prompt: "I have too many whitespace missing values in my dataset, how do I remove them?"
open_data = open_data.applymap(lambda x: x.strip() if isinstance(x, str) else x)  # Remove leading/trailing spaces
open_data.replace('', pd.NA, inplace=True)  # Replace empty strings with NaN
open_data = open_data.dropna()

open_data.columns = ["RA", "Dec", "Diam", "Dist", "Age (log years)", "Metallicity", "E(B-V)", "Cname", "_RA.icrs", "_DE.icrs"]

print("Globular Data","\n")
print(glob_data.head())

print("Open Data", "\n")
print(open_data)

Globular Data 

     Cname    RA1  RA2   RA3  Dec1    Dec2   Dec3  R_sun  E(B-V)   r_h
0   NGC104   5.67  -72   4.0  52.6  305.89 -44.89    4.5    0.04  3.17
1   NGC288  45.24  -26  34.0  57.4  152.30 -89.38    8.9    0.03  2.23
2   NGC362  14.26  -70  50.0  55.6  301.53 -46.25    8.6    0.05  0.82
4  NGC1261  16.21  -55  12.0  58.4  270.54 -52.12   16.3    0.01  2.99
5     Pal1  20.04   79  34.0  51.8  130.06  19.03   11.1    0.15  3.51
Open Data 

          RA     Dec  Diam  Dist Age (log years) Metallicity E(B-V)  \
4    00 01.7  -30 12  89.0   190            7.70        0.03   0.09   
26   00 39.4  +85 04  13.0  1550            9.70       -0.06   0.04   
42   01 26.1  +63 03   4.4   900            9.10       -0.76   0.90   
60   01 54.8  +37 26  49.0   400            9.04       -0.21   0.02   
86   02 38.8  +42 34  35.0   440            8.29       -0.26   0.04   
..       ...     ...   ...   ...             ...         ...    ...   
909  20 32.5  +28 08  31.0   800            9.04 

/tmp/ipykernel_2425789/3114775391.py:23: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  glob_data1 = pd.read_csv("glob_data.txt", skiprows = 72, nrows = 157, delim_whitespace = True, header = None)
/tmp/ipykernel_2425789/3114775391.py:24: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  glob_data2 = pd.read_csv("glob_data.txt", skiprows = 252, nrows = 157, delim_whitespace = True, header = None)
/tmp/ipykernel_2425789/3114775391.py:25: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  glob_data3 = pd.read_csv("glob_data.txt", skiprows = 433, nrows = 157, delim_whitespace = True, header = None)
/tmp/ipykernel_2425789/3114775391.py:39: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map

### Calculate the true diameter (in parsecs)

In [4]:
glob_data['true diam'] = get_true_diameter(get_angular_size(glob_data['r_h']), 1000 * glob_data['R_sun'])
glob_data

,Cname,RA1,RA2,RA3,Dec1,Dec2,Dec3,R_sun,E(B-V),r_h,true diam
0,NGC104,5.67,-72,4.0,52.6,305.89,-44.89,4.5,0.04,3.17,20.747729
1,NGC288,45.24,-26,34.0,57.4,152.30,-89.38,8.9,0.03,2.23,28.866469
2,NGC362,14.26,-70,50.0,55.6,301.53,-46.25,8.6,0.05,0.82,10.256781
4,NGC1261,16.21,-55,12.0,58.4,270.54,-52.12,16.3,0.01,2.99,70.885530
5,Pal1,20.04,79,34.0,51.8,130.06,19.03,11.1,0.15,3.51,56.666826
...,...,...,...,...,...,...,...,...,...,...,...
152,NGC7089,27.02,0,49.0,23.7,53.37,-35.77,11.5,0.06,1.06,17.729746
153,NGC7099,22.12,-23,10.0,47.5,27.18,-46.84,8.1,0.03,1.03,12.134476
154,Pal12,38.84,-21,15.0,9.4,30.51,-47.68,19.0,0.02,3.64,100.589762
155,Pal13,44.44,12,46.0,19.2,87.10,-42.70,26.0,0.05,0.36,13.613652


## Logistic Regression

In [50]:
## Necessary imports
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score
from sklearn import metrics
import sklearn as sk

In [42]:
## setting up data for logistic regression
glob_data['cluster_type'] = 1 #making globular clusters equal to one
open_data['cluster_type'] = 0 #making open clusters equal to zero

In [7]:
## Drop unwanted columns
glob_data_dropped = glob_data.drop(columns = ['Cname', 'RA1', 'RA2', 'RA3', 'Dec1', 'Dec2', 'Dec3', 'R_sun', 'r_h'])
open_data_dropped = open_data.drop(columns = ['RA', 'Dec', 'Dist', 'Age (log years)', 'Metallicity', 'Cname', '_RA.icrs', '_DE.icrs'])

## Rename diameter columns 
glob_data_dropped.rename(columns = {'true diam': 'diameter (pc)'}, inplace = True)
open_data_dropped.rename(columns = {'Diam': 'diameter (pc)'}, inplace = True)

## Combine datasets
clusters_df = pd.concat([glob_data_dropped, open_data_dropped])

In [8]:
clusters_df

,E(B-V),diameter (pc),cluster_type
0,0.04,20.747729,1
1,0.03,28.866469,1
2,0.05,10.256781,1
4,0.01,70.88553,1
5,0.15,56.666826,1
...,...,...,...
909,0.24,31.0,0
940,0.27,24.0,0
948,0.18,4.3,0
961,0.96,5.0,0


In [11]:
## make sure there are no objects
clusters_df = clusters_df.apply(pd.to_numeric, errors = 'raise')

In [12]:
# checking for infinite or nans
print(clusters_df.isna().sum())

print(np.isinf(clusters_df).sum())

pd.set_option('display.max_rows', None)

#print(cluster_df['diameter']
print(glob_data['true diam'])
print(glob_data.isna().sum())

E(B-V)           0
diameter (pc)    0
cluster_type     0
dtype: int64
E(B-V)           0
diameter (pc)    0
cluster_type     0
dtype: int64
0        20.747729
1        28.866469
2        10.256781
4        70.885530
5        56.666826
6         5.380011
7      1268.425845
8       160.617822
9         8.975411
10       12.195563
11       45.553374
12      106.922437
15       11.170176
17        1.345366
18       22.093096
19       17.390859
21       13.474025
22       17.377769
23       33.917774
24       22.621061
25       28.798110
26       34.105398
27       66.052392
28       37.815700
29       34.269751
30       12.422457
32       53.523760
33      133.047268
34       20.362300
35       46.757659
36       21.009530
37       92.119045
38       27.816356
39       19.307824
40       45.804994
41       13.721281
43       51.580615
45      135.743819
46        8.872145
47       13.855091
48       36.957575
49       29.902038
50       68.602043
51       21.706212
52       16.103670
54   

In [13]:
## Creating features and labels to train model
clust = clusters_df
features = clust.iloc[:,0:2]
label = clust['cluster_type']

In [14]:
## Defining model
train_vectors, test_vectors, train_labels, test_labels = sk.model_selection.train_test_split(features, label, test_size = 0.25, train_size = 0.75, random_state = 0)
# make sure same size
print(len(train_vectors))
print(len(test_vectors))
print(len(train_labels))
print(len(test_labels))

162
54
162
54


In [34]:
## Training model
logit_model = sm.Logit(train_labels, sm.add_constant(train_vectors))
result = logit_model.fit()
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.599905
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:           cluster_type   No. Observations:                  162
Model:                          Logit   Df Residuals:                      159
Method:                           MLE   Df Model:                            2
Date:                Wed, 13 Nov 2024   Pseudo R-squ.:                 0.08523
Time:                        16:24:36   Log-Likelihood:                -97.185
converged:                       True   LL-Null:                       -106.24
Covariance Type:            nonrobust   LLR p-value:                 0.0001169
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const            -0.2915      0.295     -0.988      0.323      -0.870       0.287
E(B-V)            2.

In [51]:
## Predict data
prediction = result.predict(sm.add_constant(test_vectors))

## Accuracy of prediction
accuracy = accuracy_score(test_labels, prediction)
print(f'Accuracy: {accuracy}')

ValueError: Classification metrics can't handle a mix of binary and continuous targets